# Grover's Algorithm: Cracking a 3-bit Password

## Problem
A secure system uses a 3-bit password (`000` to `111`). We do not know the password, but we have a black-box oracle that tells us if a guess is correct. The goal is to find the password using as few checks as possible.

## Theory
- **Classical search:** May require up to 8 checks.
- **Grover's Algorithm:** Can find the password in ~2 oracle calls (O(√8)).

Grover's algorithm works by amplifying the probability of the correct answer using the following steps:
- Create a superposition of all possible passwords.
- Use an oracle to flip the sign of the correct password's amplitude.
- Apply the diffusion operator to amplify the probability.
- Repeat for ⌊π/4 × √N⌋ times.
- Measure the system — with high probability, we get the correct password.

## Implementation

We'll use [Qiskit](https://qiskit.org/) to simulate the quantum circuit. You can run this notebook locally or in IBM Quantum Lab.

In [ ]:
# Install Qiskit if running locally
# !pip install qiskit
from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit.quantum_info import Statevector
import matplotlib.pyplot as plt
import numpy as np

### Define the 3-bit Password Oracle
Suppose the password is `x`, e.g., `x = 5` (binary `101`). The oracle will flip the sign only for `|x⟩`.

In [ ]:
def password_oracle(n_qubits, password):
    '''Creates a quantum oracle that flips the sign of the state |password⟩'''
    qc = QuantumCircuit(n_qubits)
    # Apply X to qubits where password bit is 0 (to map |password⟩ to |111⟩)
    for i in range(n_qubits):
        if ((password >> i) & 1) == 0:
            qc.x(i)
    # Multi-controlled Z (on all qubits)
    qc.h(n_qubits-1)
    qc.mct(list(range(n_qubits-1)), n_qubits-1) # multi-controlled Toffoli
    qc.h(n_qubits-1)
    # Undo the X gates
    for i in range(n_qubits):
        if ((password >> i) & 1) == 0:
            qc.x(i)
    oracle_gate = qc.to_gate()
    oracle_gate.name = f'Oracle({password:03b})'
    return oracle_gate

### Grover Diffusion Operator
Amplifies the probability of the marked state.

In [ ]:
def grover_diffusion(n_qubits):
    qc = QuantumCircuit(n_qubits)
    qc.h(range(n_qubits))
    qc.x(range(n_qubits))
    qc.h(n_qubits-1)
    qc.mct(list(range(n_qubits-1)), n_qubits-1)
    qc.h(n_qubits-1)
    qc.x(range(n_qubits))
    qc.h(range(n_qubits))
    return qc.to_gate(label='Diffusion')

### Grover's Search Circuit for 3 bits

In [ ]:
n_qubits = 3
password = 5 # Try changing this (0-7)

# Number of Grover iterations: round(pi/4 * sqrt(N)), N=8
N = 2**n_qubits
k = int(np.floor(np.pi/4 * np.sqrt(N)))
print(f'Grover iterations: {k}')

qc = QuantumCircuit(n_qubits, n_qubits)
# Step 1: Initialize superposition
qc.h(range(n_qubits))
# Step 2: Repeat Grover iteration
oracle = password_oracle(n_qubits, password)
diff = grover_diffusion(n_qubits)
for _ in range(k):
    qc.append(oracle, range(n_qubits))
    qc.append(diff, range(n_qubits))
# Step 3: Measurement
qc.measure(range(n_qubits), range(n_qubits))
qc.draw('mpl')

### Simulate the Circuit

In [ ]:
backend = Aer.get_backend('qasm_simulator')
shots = 1024
job = execute(qc, backend=backend, shots=shots)
result = job.result()
counts = result.get_counts()
plot_histogram(counts)
plt.show()
print(f'Highest probability state: {max(counts, key=counts.get)} (password: {password:03b})')

## Success Probability vs. Number of Iterations

Let's see how the success probability changes if we use 1, 2, or 3 Grover iterations.

In [ ]:
probs = []
iters = [1,2,3]
for k in iters:
    qc = QuantumCircuit(n_qubits, n_qubits)
    qc.h(range(n_qubits))
    oracle = password_oracle(n_qubits, password)
    diff = grover_diffusion(n_qubits)
    for _ in range(k):
        qc.append(oracle, range(n_qubits))
        qc.append(diff, range(n_qubits))
    qc.measure(range(n_qubits), range(n_qubits))
    job = execute(qc, backend=backend, shots=shots)
    counts = job.result().get_counts()
    success = counts.get(f'{password:03b}', 0)/shots
    probs.append(success)
plt.plot(iters, probs, 'o-')
plt.xlabel('Number of Grover Iterations')
plt.ylabel('Success Probability')
plt.title('Grover Success Probability vs Iterations')
plt.show()

# Conclusion

- Grover's algorithm finds the password with high probability in only ~2 oracle calls, compared to 8 classically.
- For a 3-bit password, the correct answer is found with >90% probability after 2 iterations.
- This demonstrates the quantum speedup for unstructured search problems.

---

Try changing `password` and number of qubits to see how Grover's algorithm scales!